In [6]:
%load_ext autoreload
%autoreload 2
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
from config import *
import utils
from tqdm import tqdm
from datetime import datetime, timedelta
import db_utils
from bson.objectid import ObjectId
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
client = MongoClient(f"mongodb+srv://{DB_USER}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority")

db = client[DB_NAME]

db_utils.init(db)

-- DB initialized -- 


In [10]:
available_rooms = json.load(open("ewa_rooms.json", "r"))

In [11]:

start_datetime = datetime.now()
end_datetime = start_datetime + timedelta(days=50)

start_date = start_datetime.strftime("%Y-%m-%dT%H:%M:%S")
end_date = end_datetime.strftime("%Y-%m-%dT%H:%M:%S")

new_events = utils.parse_all_rooms_events(available_rooms, start_date, end_date)

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [02:07<00:00,  4.10s/it]


In [19]:
new_events = utils.populate_events_room(db, new_events)

100%|██████████| 93/93 [00:00<00:00, 79492.62it/s]


In [22]:
utils.create_event_bookings(db, new_events)

100%|██████████| 93/93 [00:00<00:00, 104464.45it/s]

No new bookings to create
